# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [3]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [4]:
print(documents[0][:100])

﻿
The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [5]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [6]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [7]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

OpenAI API Key:  ········


### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


##### ✅ Answer:
1. Yes, the text-embedding-3-small is one of the newer models that have a parameter to control the overall size. By default the length of the embedding vector is 1536, but it can be reduced without losing its concept-representing properties through the dimensions parameter. 
2. OpenAI uses text embeddings to represent the semantic meaning of text strings as vectors of floating-point numbers, where the distance between vectors reflects relatedness (shorter distance = more related, bigger distance = less related). Embeddings are commonly used for search, clustering, recommendations, anomaly detection, diversity measurement, and classification. Larger embeddings generally consume more compute, memory, and storage, while smaller embeddings are more efficient. The dimensions parameter allows developers to trade off between embedding size, performance, and cost, shortening embeddings without losing their core semantic properties.


We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### ✅ Answer: Depending on the task and the amount of data, it’s better to choose async for larger or more complex embedding tasks. Sync runs each embedding one after another, while async allows multiple embeddings to be requested at the same time, collecting results as they finish. Async is faster for large datasets, reduces overall runtime, uses resources more efficiently, and can lower costs. It also improves scalability and is especially helpful for real-time applications, like search or recommendations, or when multiple users are making requests at the same time. 

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [9]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer:
The content generated from a model is non-deterministic, but there are techniques to achieve more consistent results. Pinning your production application to specific model snapshots helps ensure stable behavior over time. You can also build evaluations that measure the behavior of your prompts so you can monitor performance as you iterate or when you upgrade models. Additionally, you can give the model high-level instructions through the API (such as tone, goals, or examples of correct responses). These developer messages take priority over user input and help enforce consistency. Another technique is to use reusable prompts defined in the dashboard rather than hardcoding them, which allows you to improve and deploy updated prompts without changing your integration code.


### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [10]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [11]:
print(response)

Hello! The best way to write a loop in Python depends on what you want to achieve, but generally, using a `for` loop is clear and Pythonic for iterating over sequences like lists, strings, or ranges. Here's an example of a simple `for` loop:

```python
for i in range(5):
    print(i)
```

This will print numbers from 0 to 4.

If you need to loop while a certain condition is true, a `while` loop might be more appropriate:

```python
count = 0
while count < 5:
    print(count)
    count += 1
```

If you want, I can help you with more specific examples or best practices depending on your task. Just let me know! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [12]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [13]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [14]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: The "Michael Eisner Memorial Weak Executive Problem" refers to the phenomenon where a CEO, who was formerly successful in a particular function (such as product management, sales, or marketing), ends up hiring a weak executive to run that same function after the CEO has moved on to a broader role. This happens because the CEO has a hard time letting go of the function that brought them success and wants to remain "the man" in that area. The example given is Michael Eisner, the CEO of Disney, who had previously been a brilliant TV network executive. When he bought ABC, the network dropped to fourth place, but Eisner claimed that if he had extra time, he could have turned ABC around himself. However, he did not, illustrating how hiring a weak executive for the CEO's former specialty can harm that function (Source 1).

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2: 0.509', 'Source 3: 0.479']


#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:
To have the LLM create more thoughtful and detailed responses, there are several prompting strategies that can help (based on reasoning-based prompting to encourage the model to “think” more deeply).First, chain-of-thought prompting, where you ask the model to think step-by-step or explain its reasoning before answering. Another option is deliberation prompting, which encourages the model to spend more time reasoning and produce deeper, more structured outputs. This is also where you can change the reasoning parameter to control how much computational effort the model puts into reasoning.Instructional scaffolding helps break the request into stages (for example: first list these factors, then analyze the list, then conclude with recommendations). Role prompting can change the level of depth and framing by asking the model to answer as an expert, researcher, or advisor. Lastly, self-critique (reflection) asks the model to give an answer and then review it for weaknesses and ways to improve.


### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database
- Use a different embedding model
- Add the capability to ingest a YouTube link

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

In [ ]:

import os
import re
import uuid
import time
import numpy as np
from getpass import getpass
from typing import List, Dict, Tuple, Any
import asyncio
from concurrent.futures import ThreadPoolExecutor

# OpenAI client
from openai import OpenAI

# Document loaders
import fitz  # PyMuPDF (runs on local, issues with Conda)
import docx  # python-docx

# For running in notebooks without "event loop already running" errors:
try:
    import nest_asyncio
    nest_asyncio.apply()
except Exception:
    # not critical in plain scripts
    pass

###OpenAI API key 
def set_openai_api_key(api_key: str = None) -> OpenAI:
    """
    Sets OPENAI_API_KEY env var and returns an initialized OpenAI client.
    """
    if api_key is None:
        api_key = getpass("OpenAI API Key: ")
    os.environ["OPENAI_API_KEY"] = api_key
    # instantiate client AFTER setting env var
    return OpenAI()

###can load both text and PDF documents, vs. the original which was only .txt
class TextLoader:
    def __init__(self, path: str, encoding: str = "utf-8"):
        self.path = path
        self.encoding = encoding

    def load(self) -> str:
        with open(self.path, "r", encoding=self.encoding) as f:
            return f.read()

class PDFLoader:
    def __init__(self, path: str):
        self.path = path

    def load(self) -> str:
        text_parts = []
        doc = fitz.open(self.path)
        for pno in range(doc.page_count):
            page = doc.load_page(pno)
            txt = page.get_text("text")
            text_parts.append(f"\n\n[PAGE {pno+1}]\n{txt}")
        doc.close()
        return "\n".join(text_parts)

class DOCXLoader:
    def __init__(self, path: str):
        self.path = path

    def load(self) -> str:
        doc = docx.Document(self.path)
        paragraphs = [p.text for p in doc.paragraphs if p.text.strip() != ""]
        return "\n\n".join(paragraphs)


###### This changes the way that the text is split. In the original it did.., whereas this add components to make it more sentence and paragraph awarness


class SimpleTextSplitter:
    def __init__(self, chunk_size: int = 800, chunk_overlap: int = 100):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.sentence_end_re = re.compile(r'(?<=[.!?])\s+')

    def split_text(self, text: str) -> List[str]:
        sentences = [s.strip() for s in self.sentence_end_re.split(text) if s.strip()]
        chunks = []
        current = ""
        for s in sentences:
            if len(current) + len(s) + 1 <= self.chunk_size:
                current = (current + " " + s).strip()
            else:
                if current:
                    chunks.append(current)
                if len(s) > self.chunk_size:
                    for i in range(0, len(s), self.chunk_size - self.chunk_overlap):
                        part = s[i:i + (self.chunk_size - self.chunk_overlap)]
                        chunks.append(part)
                    current = ""
                else:
                    current = s
        if current:
            chunks.append(current)

        if self.chunk_overlap > 0 and len(chunks) > 1:
            overlapped = []
            for i, c in enumerate(chunks):
                if i == 0:
                    overlapped.append(c)
                else:
                    prev = overlapped[-1]
                    overlap = prev[-self.chunk_overlap:] if len(prev) > self.chunk_overlap else prev
                    overlapped.append((overlap + " " + c).strip())
            return overlapped
        return chunks


class VectorDatabase:
    def __init__(self, client: OpenAI, embeddings_model: str = "text-embedding-3-large"):
        self.records: List[Dict[str, Any]] = []
        self.embeddings_model = embeddings_model
        self._norm_cache = None
        self.client = client

    def insert(self, text: str, embedding: np.ndarray, metadata: Dict[str, Any]):
        rec = {
            "id": str(uuid.uuid4()),
            "text": text,
            "embedding": np.asarray(embedding, dtype=np.float32),
            "meta": metadata
        }
        self.records.append(rec)
        self._norm_cache = None

    def _ensure_norms(self):
        if self._norm_cache is None:
            if not self.records:
                self._norm_cache = np.empty((0, 0), dtype=np.float32)
                return
            mats = np.vstack([r["embedding"] for r in self.records])
            norms = np.linalg.norm(mats, axis=1, keepdims=True)
            norms[norms == 0] = 1.0
            self._norm_cache = mats / norms

    def search(self, query_embedding: np.ndarray, k: int = 4) -> List[Tuple[Dict[str, Any], float]]:
        if len(self.records) == 0:
            return []
        self._ensure_norms()
        q = np.asarray(query_embedding, dtype=np.float32)
        qnorm = q / (np.linalg.norm(q) + 1e-12)
        scores = np.dot(self._norm_cache, qnorm)
        top_idx = np.argsort(-scores)[:k]
        results = [(self.records[int(i)], float(scores[int(i)])) for i in top_idx]
        return results

    async def abuild_from_texts(self, list_of_texts: List[Tuple[str, Dict[str, Any]]], batch_size: int = 32):
        loop = asyncio.get_running_loop()
        with ThreadPoolExecutor(max_workers=4) as pool:
            for i in range(0, len(list_of_texts), batch_size):
                batch = list_of_texts[i:i + batch_size]
                texts = [t for t, _ in batch]
                metas = [m for _, m in batch]
                # call the client's embeddings endpoint in a thread
                def _embed_call():
                    return self.client.embeddings.create(model=self.embeddings_model, input=texts)
                try:
                    embeddings_resp = await loop.run_in_executor(pool, _embed_call)
                except Exception as e:
                    raise RuntimeError(f"Embedding API call failed: {e}")
                # align embeddings explicitly by index (safer than pop)
                for j, item in enumerate(embeddings_resp.data):
                    emb = np.array(item.embedding, dtype=np.float32)
                    text = texts[j]
                    meta = metas[j]
                    self.insert(text, emb, meta)
                # be cooperative with rate limits
                await asyncio.sleep(0.05)


def load_and_chunk_files(file_paths: List[str], splitter: SimpleTextSplitter) -> List[Tuple[str, Dict[str, Any]]]:
    all_chunks = []
    for path in file_paths:
        lower = path.lower()
        if lower.endswith(".pdf"):
            loader = PDFLoader(path)
        elif lower.endswith(".docx"):
            loader = DOCXLoader(path)
        else:
            loader = TextLoader(path)
        raw_text = loader.load()
        chunks = splitter.split_text(raw_text)
        total = len(chunks)
        for i, c in enumerate(chunks):
            meta = {"source": os.path.basename(path), "chunk_index": i, "total_chunks": total}
            all_chunks.append((c, meta))
    return all_chunks

##### RAG pipeline that add in self-reflection and transparency of references
class EnhancedRAG:
    def __init__(self, client: OpenAI, llm_model: str = "gpt-4o-mini", embeddings_model: str = "text-embedding-3-large"):
        self.client = client
        self.llm_model = llm_model
        self.vector_db = VectorDatabase(client=client, embeddings_model=embeddings_model)

    async def ingest_files(self, file_paths: List[str], splitter: SimpleTextSplitter):
        corpus = load_and_chunk_files(file_paths, splitter)
        await self.vector_db.abuild_from_texts(corpus)

    def _build_context_block(self, results: List[Tuple[Dict[str, Any], float]]) -> Tuple[str, List[str]]:
        blocks = []
        source_ids = []
        for i, (rec, score) in enumerate(results, start=1):
            sid = f"SRC-{i}"
            source_ids.append(sid)
            meta = rec["meta"]
            snippet = rec["text"][:1000].replace("\n", " ").strip()
            blocks.append(f"[{sid}] Source: {meta.get('source','unknown')} - chunk {meta.get('chunk_index')}/{meta.get('total_chunks')}\n\n{snippet}\n")
        context_text = "\n\n".join(blocks)
        return context_text, source_ids

    def _call_llm(self, messages: List[Dict[str, str]], temperature: float = 0.0, max_tokens: int = 800) -> str:
        try:
            resp = self.client.chat.completions.create(model=self.llm_model, messages=messages, temperature=temperature, max_tokens=max_tokens)
            return resp.choices[0].message.content
        except Exception as e:
            raise RuntimeError(f"LLM call failed: {e}")

    def ask(self, user_query: str, k: int = 4, do_self_reflection: bool = True) -> Dict[str, Any]:
        try:
            emb_resp = self.client.embeddings.create(model=self.vector_db.embeddings_model, input=[user_query])
        except Exception as e:
            raise RuntimeError(f"Embedding call failed for query: {e}")
        q_emb = np.array(emb_resp.data[0].embedding, dtype=np.float32)
        results = self.vector_db.search(q_emb, k=k)
        context_text, source_ids = self._build_context_block(results)

        system_msg = {
            "role": "system",
            "content": "You are a helpful assistant answering strictly from the context. Cite every factual claim using (SRC-n). If unsupported, say 'UNSUPPORTED'."
        }
        user_msg = {
            "role": "user",
            "content": f"Context:\n\n{context_text}\n\nQuestion: {user_query}\n\nAnswer using only the context and cite inline (SRC-n). Then list sources used."
        }
        first_answer = self._call_llm([system_msg, user_msg])

        output = {"first_answer": first_answer, "retrieved": [(r['meta'], s) for (r, s) in results]}

        if do_self_reflection:
            verify_system = {"role": "system", "content": "You verify claims against context; be conservative."}
            verify_user = {"role": "user", "content": f"Context:\n\n{context_text}\n\nPrevious answer:\n\n{first_answer}\n\nTask:\n1) Number claims.\n2) For each claim, list supporting SRC tags or UNSUPPORTED.\n3) Produce a revised final answer retaining only supported claims. Return JSON with keys: claims, verification, final_answer."}
            verification_response = self._call_llm([verify_system, verify_user], temperature=0.0, max_tokens=1000)
            output["verification"] = verification_response

            summarizer_system = {"role": "system", "content": "Produce a concise final answer and confidence (0-100)."}
            summarizer_user = {"role": "user", "content": f"Verification output:\n{verification_response}\n\nProduce JSON: {{'final_answer_short':..., 'confidence': <int>}}"}
            summary_response = self._call_llm([summarizer_system, summarizer_user], temperature=0.0, max_tokens=200)
            output["final_summary"] = summary_response

        return output

############### Example
async def demo_pmarca_rag():
    """Demo with PMarcaBlogs.txt and original questions"""
    print("🏢 PMARCA BLOGS RAG DEMO")
    print("=" * 50)
    
    client = set_openai_api_key()  # prompts for key, returns OpenAI client
    files = ["data/PMarcaBlogs.txt"]  # Original text file
    splitter = SimpleTextSplitter(chunk_size=800, chunk_overlap=120)
    rag = EnhancedRAG(client=client, llm_model="gpt-4o-mini", embeddings_model="text-embedding-3-large")

    print(f"📄 Loading: {files[0]}")
    await rag.ingest_files(files, splitter)
    print(f"✅ Loaded {len(rag.vector_db.records)} chunks")
    print("🎉 Vector database ready!")

    # Original questions from the PMarca blogs
    questions = [
        "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
        "What are Marc Andreessen's views on software and technology?",
        "What does he say about startups and entrepreneurship?",
        "What are his thoughts on the future of the internet?",
        "What advice does he give to entrepreneurs and founders?"
    ]

    print("\n🎯 ANSWERING 5 ORIGINAL QUESTIONS")
    print("=" * 50)

    for i, question in enumerate(questions, 1):
        print(f"\n{'='*50}")
        print(f"QUESTION {i}: {question}")
        print('='*50)
        
        try:
            print(f"🔍 Searching for: {question}")
            result = rag.ask(question, k=4, do_self_reflection=True)
            print(f"📊 Found {len(result['retrieved'])} relevant chunks")
            print(f"📝 ANSWER: {result['first_answer']}")
            
            if result.get("verification"):
                print(f"\n🔍 VERIFICATION: {result['verification']}")
            
            if result.get("final_summary"):
                print(f"\n📊 FINAL SUMMARY: {result['final_summary']}")
                
        except Exception as e:
            print(f"❌ Error processing question: {e}")
        
        print("-" * 50)

    print("\n🎉 Analysis complete!")
    print("💡 You can now ask additional questions interactively...")

    while True:
        try:
            user_question = input("\n❓ Ask another question about PMarca blogs (or 'quit' to exit): ").strip()
            if user_question.lower() in ['quit', 'exit', 'q']:
                break
                
            if not user_question:
                continue
                
            print("🔍 Searching and analyzing...")
            result = rag.ask(user_question, k=4, do_self_reflection=True)
            
            print(f"\n📝 ANSWER: {result['first_answer']}")
            if result.get("final_summary"):
                print(f"\n📊 SUMMARY: {result['final_summary']}")
                
        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"❌ Error: {e}")

async def demo_nestle_rag():
    """Demo with Nestle HR PDF"""
    print("🏢 NESTLE HR POLICY RAG DEMO")
    print("=" * 50)
    
    client = set_openai_api_key()  # prompts for key, returns OpenAI client
    files = ["data/the_nestle_hr_policy_pdf_2012.pdf"]  # adjust path
    splitter = SimpleTextSplitter(chunk_size=800, chunk_overlap=120)
    rag = EnhancedRAG(client=client, llm_model="gpt-4o-mini", embeddings_model="text-embedding-3-large")

    print(f"📄 Loading: {files[0]}")
    await rag.ingest_files(files, splitter)
    print(f"✅ Loaded {len(rag.vector_db.records)} chunks")
    print("🎉 Vector database ready!")

    questions = [
        "What is Nestle's approach to employee recruitment and hiring?",
        "What are the main HR policies and principles outlined in this document?",
        "What are the key employee benefits and compensation policies?",
        "What are the company's policies on diversity and inclusion?",
        "How does Nestle handle employee performance management and development?",
    ]

    print("\n🎯 ANSWERING 5 NESTLE QUESTIONS")
    print("=" * 50)

    for i, question in enumerate(questions, 1):
        print(f"\n{'='*50}")
        print(f"QUESTION {i}: {question}")
        print('='*50)
        
        try:
            print(f"🔍 Searching for: {question}")
            result = rag.ask(question, k=4, do_self_reflection=True)
            print(f"📊 Found {len(result['retrieved'])} relevant chunks")
            print(f"📝 ANSWER: {result['first_answer']}")
            
            if result.get("final_summary"):
                print(f"\n📊 FINAL SUMMARY: {result['final_summary']}")
                
        except Exception as e:
            print(f"❌ Error processing question: {e}")
        
        print("-" * 50)

    print("\n🎉 Analysis complete!")

if __name__ == "__main__":
    import sys
    
    # Choose which demo to run
    if len(sys.argv) > 1 and sys.argv[1] == "nestle":
        asyncio.run(demo_nestle_rag())
    else:
        print("Running PMarca Blogs demo...")
        print("(Use 'python new_rag.py nestle' for Nestle HR policy demo)")
        asyncio.run(demo_pmarca_rag())
